In [26]:
import requests
from datetime import datetime
import json
import time
import pandas as pd
from time import mktime

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [27]:
class PolgonData(object):

    def __init__(self):
        self.params=params = (('apiKey', 'M_PKVL_rqHZI7VM9ZYO_hwPiConz5rIklx893F'),)

    def PolygonLastTrades(self,symbol):
        # Make use of Tickers
        requesturl='https://api.polygon.io/v1/last/stocks/'+symbol
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

    def PolygonHistoricTrades(self, date=None, symbol=None,startTS=None,endTS=None,limitresult=10):
        if startTS:
            # For Getting Paginated Request
            requesturl='https://api.polygon.io/v2/ticks/stocks/nbbo/'+symbol+'/'+date+'?timestamp='+startTS+'&timestampLimit='+endTS+'&limit='+limitresult
            print("Paginated Request For = " + symbol)
        else:
            requesturl='https://api.polygon.io/v2/ticks/stocks/nbbo/'+symbol+'/'+date+'?timestampLimit='+endTS+'&limit='+limitresult
            print("First Request For = " + symbol)
        print(requesturl)
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)
    
    def PolygonDailyOpenClose(self,date=None, symbol=None):
        requesturl='https://api.polygon.io/v1/open-close/'+symbol+'/'+date
        print(requesturl)
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)
    
    def PolygonAggregdateData(self):
        # Make use of Tickers, Date and Limit
        requesturl='https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2020-02-14/2020-02-15'
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

In [28]:
from datetime import datetime

class DateTimeManipulation(object):
    
    def __init__(self,date=None):
        self.date=date

    # Returns timestamp with milliseconds
    def unix_time_millis(self,dt):
        epoch = datetime.utcfromtimestamp(0)
        tsDate=(dt - epoch).total_seconds() * 1000.0
        tsDate=str(int(tsDate))+'000000'
        return tsDate

    def stringTimeToDatetime(self,date=None,time=None):
        marketOpenTSStr = date +' '+ time
        return datetime.strptime(marketOpenTSStr,'%Y-%m-%d %H:%M:%S')
    
    def convertStringDateToTS(self,starttime='9:30:00',endtime='17:00:00'):
        marketOpenTSStr = self.date +' '+ starttime
        marketCloseTSStr = self.date +' ' + endtime
        
        marketTimeStamps={}
        marketTimeStamps['marketOpenTS']=self.unix_time_millis(datetime.strptime(marketOpenTSStr,'%Y-%m-%d %H:%M:%S'))
        marketTimeStamps['marketCloseTS']=self.unix_time_millis(datetime.strptime(marketCloseTSStr,'%Y-%m-%d %H:%M:%S'))
        return marketTimeStamps
    
    def getHumanTime(self,ts,getMilliSecondsAlso=False):
        try:
            s, ms = divmod(ts, 1000000000)
            if getMilliSecondsAlso:
                return datetime(*time.gmtime(s)[:6]),ms
            else:
                return datetime(*time.gmtime(s)[:6])
            #print('{}.{:03d}'.format(time.strftime('%Y-%m-%d %H:%M:%S',  time.gmtime(s)), ms))
        except AttributeError:
            print("Attribute Error Occured")
            print(ts)
            print(s)
            print(ms)
            

In [29]:
# Taking in ETF List
holdings=pd.read_csv("XLK-holdings20200220.csv")

holdings['Weighting']=holdings['Weighting'].apply(lambda x:x.replace('%',''))
holdings['Weighting']=holdings['Weighting'].astype(float)
holdings['Weighting']=holdings['Weighting']/100
weights=dict(zip(holdings.Symbol,holdings.Weighting))

cashvalue=holdings[holdings['Symbol']=='CASH'].get('Weighting').item()*28583351000

symbols=list(holdings['Symbol'].values)+['XLK']
symbols.remove('CASH')

# Process the ticker data


XLK: Technology Select Sector SPDR Fund <br>
Inception Date: 1998-12-16 <br>
Fund Holdings as of: 2020-02-21 <br>
"Total Assets Under Management (in thousands):	28583351" <br>
Shares Outstanding: 287356000 <br>
Expense Ratio: 0.13% <br>
Tracks This Index: Technology Select Sector Index <br>
ETFdb.com Category: Technology Equities <br>
Issuer: State Street SPDR <br>
Structure: ETF <br>

In [33]:
# Create an object of date when we need and time between which we need data
previousdate='2020-02-20'
date='2020-02-21'
starttime='9:30:00'
endtime='17:00:00'
endtimeLoop='16:00:00'

class GetETFFrame(object):
    
    def __init__(self):
        self.tickHistData={}
        self.date=date
        self.starttime=starttime
        self.endtime=endtime
        self.endtimeLoop=endtimeLoop
        self.extractDataTillTime = DateTimeManipulation().stringTimeToDatetime(date=self.date,time=self.endtimeLoop)
        self.datetimeObj=DateTimeManipulation(self.date)
        self.marketTimeStamps=self.datetimeObj.convertStringDateToTS(starttime=self.starttime,endtime=self.endtime)
        print(self.starttime)
        print(self.endtime)
        print(self.marketTimeStamps)

    def getDataFromPolygon(self,symbol):
        # First Request
        data=PolgonData().PolygonHistoricTrades(date=self.date,symbol=symbol,endTS=self.marketTimeStamps['marketCloseTS'],limitresult=str(50000))
        # Last timestamp from data received
        lastUnixTimeStamp = data['results'][-1]['t']
        # Covert UNIX timestamp to human timestamp
        lastHumanTimeStamp = self.datetimeObj.getHumanTime(lastUnixTimeStamp)
        # Get timestamp for date +  '18:00:00' hrs - Make use of pagination
        # Paginated Request if the data from above doesn't reach 5 pm time
        while lastHumanTimeStamp < self.extractDataTillTime:
            print(self.datetimeObj.getHumanTime(data['results'][-1]['t']))
            data2=PolgonData().PolygonHistoricTrades(date=self.date,symbol=symbol,startTS=str(lastUnixTimeStamp),endTS=self.marketTimeStamps['marketCloseTS'],limitresult=str(50000))
            # Last timestamp from data received
            lastUnixTimeStamp = data2['results'][-1]['t']
            # Covert UNIX timestamp to human timestamp
            lastHumanTimeStamp = self.datetimeObj.getHumanTime(lastUnixTimeStamp)
            # Get timestamp for date +  '18:00:00' hrs - Make use of pagination
            data['results']=data['results'] + data2['results']
        self.tickHistData[symbol] = data
    
    def returnDf(self):
        return self.tickHistData
    
import concurrent.futures    
executor = concurrent.futures.ProcessPoolExecutor(20)
ob=GetETFFrame()
#futures = [executor.submit(ob.getDataFromPolygon(symbol)) for symbol in symbols]

for symbol in symbols:
    ob.getDataFromPolygon(symbol)

#concurrent.futures.wait(futures)
tickHistData=ob.returnDf()

9:30:00
17:00:00
{'marketOpenTS': '1582277400000000000', 'marketCloseTS': '1582304400000000000'}
First Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestampLimit=1582304400000000000&limit=50000
2020-02-21 14:46:37
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582296397554225623&timestampLimit=1582304400000000000&limit=50000
2020-02-21 14:58:15
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582297095661536527&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:16:22
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582298182134773058&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:41:34
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582299694220837481&timestampLimit=1582304400000000000&limit=50000
First Request For = AAPL
ht

2020-02-21 15:21:08
Paginated Request For = HPQ
https://api.polygon.io/v2/ticks/stocks/nbbo/HPQ/2020-02-21?timestamp=1582298468853459198&timestampLimit=1582304400000000000&limit=50000
First Request For = MSI
https://api.polygon.io/v2/ticks/stocks/nbbo/MSI/2020-02-21?timestampLimit=1582304400000000000&limit=50000
First Request For = TEL
https://api.polygon.io/v2/ticks/stocks/nbbo/TEL/2020-02-21?timestampLimit=1582304400000000000&limit=50000
First Request For = APH
https://api.polygon.io/v2/ticks/stocks/nbbo/APH/2020-02-21?timestampLimit=1582304400000000000&limit=50000
First Request For = PAYX
https://api.polygon.io/v2/ticks/stocks/nbbo/PAYX/2020-02-21?timestampLimit=1582304400000000000&limit=50000
First Request For = FLT
https://api.polygon.io/v2/ticks/stocks/nbbo/FLT/2020-02-21?timestampLimit=1582304400000000000&limit=50000
First Request For = KLAC
https://api.polygon.io/v2/ticks/stocks/nbbo/KLAC/2020-02-21?timestampLimit=1582304400000000000&limit=50000
First Request For = MCHP
https:/

In [34]:
lastprieforNaFilling={}
ob=PolgonData()
for symbol in symbols:
    d=ob.PolygonDailyOpenClose(date=previousdate,symbol=symbol)
    lastprieforNaFilling[symbol]=d['close']

https://api.polygon.io/v1/open-close/MSFT/2020-02-20
https://api.polygon.io/v1/open-close/AAPL/2020-02-20
https://api.polygon.io/v1/open-close/V/2020-02-20
https://api.polygon.io/v1/open-close/MA/2020-02-20
https://api.polygon.io/v1/open-close/INTC/2020-02-20
https://api.polygon.io/v1/open-close/CSCO/2020-02-20
https://api.polygon.io/v1/open-close/ADBE/2020-02-20
https://api.polygon.io/v1/open-close/NVDA/2020-02-20
https://api.polygon.io/v1/open-close/CRM/2020-02-20
https://api.polygon.io/v1/open-close/PYPL/2020-02-20
https://api.polygon.io/v1/open-close/ACN/2020-02-20
https://api.polygon.io/v1/open-close/IBM/2020-02-20
https://api.polygon.io/v1/open-close/AVGO/2020-02-20
https://api.polygon.io/v1/open-close/TXN/2020-02-20
https://api.polygon.io/v1/open-close/ORCL/2020-02-20
https://api.polygon.io/v1/open-close/QCOM/2020-02-20
https://api.polygon.io/v1/open-close/FIS/2020-02-20
https://api.polygon.io/v1/open-close/ADP/2020-02-20
https://api.polygon.io/v1/open-close/INTU/2020-02-20
http

# Meaning of Each Columns
## Data Mapping 


 'i': {'name': 'indicators', 'type': '[]int' <br>
 's': {'name': 'bid_size', 'type': 'int'}, <br>
 'x': {'name': 'bid_exchange', 'type': 'int'}, <br>
 'P': {'name': 'ask_price', 'type': 'float64'}, <br>
 'S': {'name': 'ask_size', 'type': 'int'}, <br>
 't': {'name': 'sip_timestamp', 'type': 'int64'}, <br>
 'f': {'name': 'trf_timestamp', 'type': 'int64'}, <br>
 'c': {'name': 'conditions', 'type': '[]int'}, <br>
 'z': {'name': 'tape', 'type': 'int'}, <br>
 'X': {'name': 'ask_exchange', 'type': 'int'}, <br>
 'y': {'name': 'participant_timestamp', 'type': 'int64'}, <br>
 'q': {'name': 'sequence_number', 'type': 'int'}, <br>
 'p': {'name': 'bid_price', 'type': 'float64'} <br>
 
 ### Clean the Holdings data

In [98]:
finalDF=[]
appendData=[]
for key,value in tickHistData.items():
    df=pd.DataFrame.from_dict(value['results'])
    df['Symbol']=key
    appendData.append(df)

finalDF= pd.concat(appendData)

### Perform Analysis

In [99]:
# Clean the DataFrame
finalDF['t']=finalDF['t'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))
finalDF['y']=finalDF['y'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))
finalDF['Spread']=finalDF['P']-finalDF['p']
finalDF['MidPrice']=(finalDF['P']+finalDF['p'])/2

NameError: name 'S' is not defined

In [100]:
finalDF=finalDF[finalDF['S']!=0]
finalDF=finalDF[finalDF['s']!=0]
finalDF.head(100)

,P,S,X,c,i,p,q,s,t,x,y,z,Symbol,Spread,MidPrice
4,183.69,1,11,[1],NaN,183.59,3017,1,2020-02-21 09:00:53,12,2020-02-21 09:00:53,3,MSFT,0.10,183.640
5,183.69,1,11,[1],NaN,183.52,3050,1,2020-02-21 09:00:58,11,2020-02-21 09:00:58,3,MSFT,0.17,183.605
6,183.60,1,12,[1],NaN,183.52,4317,1,2020-02-21 09:04:58,11,2020-02-21 09:04:58,3,MSFT,0.08,183.560
7,183.65,1,11,[1],NaN,183.52,4363,1,2020-02-21 09:05:02,11,2020-02-21 09:05:02,3,MSFT,0.13,183.585
8,183.65,1,11,[1],NaN,183.55,4530,1,2020-02-21 09:05:12,12,2020-02-21 09:05:12,3,MSFT,0.10,183.600
9,183.65,1,11,[1],NaN,183.52,4954,1,2020-02-21 09:07:04,11,2020-02-21 09:07:04,3,MSFT,0.13,183.585
10,183.62,5,11,[1],NaN,183.36,5065,1,2020-02-21 09:07:41,12,2020-02-21 09:07:41,3,MSFT,0.26,183.490
11,183.62,5,11,[1],NaN,183.36,5069,1,2020-02-21 09:07:43,11,2020-02-21 09:07:43,3,MSFT,0.26,183.490
12,183.62,5,11,[1],NaN,183.38,5086,1,2020-02-21 09:07:49,12,2020-02-21 09:07:49,3,MSFT,0.24,183.500
13,183.62,5,11,[1],NaN,183.38,5093,1,2020-02-21 09:07:49,11,2020-02-21 09:07:49,3,MSFT,0.24,183.500


In [101]:
dfGroupedByObject=finalDF.groupby([finalDF['t'].dt.hour,finalDF['t'].dt.minute,finalDF['Symbol']])['MidPrice'].mean()
df=dfGroupedByObject.unstack(level=2)
df=df.fillna(method='ffill')
df=df.fillna(lastprieforNaFilling)

etfspreadgroupby=finalDF[finalDF['Symbol']=='XLK']
etfspread=etfspreadgroupby.groupby([etfspreadgroupby['t'].dt.hour,etfspreadgroupby['t'].dt.minute])['Spread'].mean()
etfspread=etfspread.fillna(0)

In [108]:
getmeHourDataFor=11
etfticker='XLK'
for name,group in df.groupby(level=0):
    if name==getmeHourDataFor:
        break

In [109]:
etfprice=group[etfticker]
del group[etfticker]

In [110]:
group=group.pct_change().dropna()*100

In [113]:
etfpricechange=etfprice.pct_change().dropna()*100
etfpricechange=etfpricechange.unstack(level=1)

In [115]:
netassetvaluereturn=group.assign(**weights).mul(group).sum(axis=1)
netassetvaluereturn=netassetvaluereturn.unstack(level=1)

In [117]:
ds=pd.concat([etfprice.unstack(level=1),etfpricechange,netassetvaluereturn],axis=0).T
ds.columns=['ETF Price','ETF Change Price %','Net Asset Value Change%']

In [118]:
ds['Arbitrage in $']=(ds['ETF Change Price %'] - ds['Net Asset Value Change%'])*ds['ETF Price']/100
ds['ETF Trading Spread in $']=etfspread.unstack(level=1).loc[getmeHourDataFor]

In [125]:
def getMeHourdata(getmeHourDataFor=None):
    etfticker='XLK'
    for name,group in df.groupby(level=0):
        if name==getmeHourDataFor:
            break

    etfprice=group[etfticker]
    del group[etfticker]

    group=group.pct_change().dropna()*100
    etfpricechange=etfprice.pct_change().dropna()*100
    etfpricechange=etfpricechange.unstack(level=1)

    netassetvaluereturn=group.assign(**weights).mul(group).sum(axis=1)
    netassetvaluereturn=netassetvaluereturn.unstack(level=1)

    ds=pd.concat([etfprice.unstack(level=1),etfpricechange,netassetvaluereturn],axis=0).T
    ds.columns=['ETF Price','ETF Change Price %','Net Asset Value Change%']

    ds['Arbitrage in $']=(ds['ETF Change Price %'] - ds['Net Asset Value Change%'])*ds['ETF Price']/100
    ds['ETF Trading Spread in $']=etfspread.unstack(level=1).loc[getmeHourDataFor]
    return ds

In [127]:
getMeHourdata(9)

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,101.073359,NaN,NaN,NaN,0.378906
1,101.095647,0.022051,2.054895,-2.055117,0.335294
2,101.084545,-0.010981,-0.003155,-0.007912,0.341091
3,101.082176,-0.002344,0.005902,-0.008335,0.367412
4,101.049939,-0.031892,-0.012687,-0.019407,0.445732
5,101.068622,0.018489,0.005528,0.013100,0.389898
6,101.080341,0.011595,-0.000333,0.012056,0.390455
7,101.061010,-0.019124,-0.016799,-0.002350,0.433535
8,101.058500,-0.002484,-0.068152,0.066363,0.455727


In [128]:
getMeHourdata(10)

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,100.946718,NaN,NaN,NaN,0.442137
1,100.990268,0.043142,-2.802111e-04,0.043852,0.464464
2,100.980490,-0.009682,5.465971e-03,-0.015296,0.394314
3,100.979592,-0.000890,5.223695e-03,-0.006173,0.386939
4,101.029000,0.048929,2.103490e-03,0.047307,0.338000
5,101.001250,-0.027467,1.179804e-03,-0.028934,0.282500
6,101.068125,0.066212,9.683931e-03,0.057132,0.416250
7,101.010833,-0.056686,3.165630e-03,-0.060457,0.301667
8,101.005000,-0.005775,-1.758339e-03,-0.004057,0.290000


In [129]:
getMeHourdata(11)

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,101.150098,NaN,NaN,NaN,0.365294
1,101.154302,0.004156,0.000316,0.003884,0.348605
2,101.153295,-0.000995,-0.001824,0.000838,0.346591
3,101.179405,0.025812,0.000105,0.026010,0.398810
4,101.060000,-0.118013,-0.000837,-0.118417,0.417746
5,101.087000,0.026717,0.000452,0.026551,0.389200
6,101.044067,-0.042472,-0.015583,-0.027170,0.379333
7,101.038696,-0.005316,0.003475,-0.008882,0.375217
8,101.082604,0.043457,-0.001022,0.044961,0.362292


In [130]:
getMeHourdata(12)

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,101.030326,NaN,NaN,NaN,0.243696
1,101.037273,6.875797e-03,0.062046,-0.055743,0.245455
2,101.107778,6.978123e-02,0.015398,0.054986,0.195556
3,101.088354,-1.921130e-02,-0.007142,-0.012200,0.163293
4,101.079647,-8.612861e-03,0.004390,-0.013143,0.178706
5,101.077957,-1.672018e-03,124.737501,-126.083808,0.164086
6,101.070714,-7.165463e-03,0.014582,-0.021981,0.175714
7,101.093017,2.206668e-02,-0.008188,0.030585,0.133966
8,101.067679,-2.506471e-02,-0.010169,-0.015054,0.171786


In [131]:
getMeHourdata(13)

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,101.306563,NaN,NaN,NaN,0.193125
1,101.317907,0.011198,0.006348,0.004914,0.215814
2,101.318929,0.001008,0.007235,-0.006309,0.217857
3,101.312586,-0.006260,0.006559,-0.012988,0.205172
4,101.312368,-0.000215,-0.005746,0.005604,0.204737
5,101.312973,0.000597,-0.005328,0.006003,0.205946
6,101.312586,-0.000382,-0.000244,-0.000139,0.205172
7,101.310600,-0.001960,-0.004628,0.002703,0.201200
8,101.312813,0.002184,-0.016311,0.018737,0.205625


In [132]:
getMeHourdata(14)

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,101.169665,NaN,NaN,NaN,0.270915
1,101.193349,2.341060e-02,0.004808,0.018825,0.229151
2,101.226333,3.259530e-02,0.019856,0.012895,0.255333
3,101.230875,4.486645e-03,0.017222,-0.012892,0.261750
4,101.218594,-1.213192e-02,-0.006881,-0.005315,0.237188
5,101.230469,1.173203e-02,0.011883,-0.000153,0.260938
6,101.232632,2.136539e-03,0.015057,-0.013080,0.265263
7,101.203468,-2.880873e-02,0.030018,-0.059535,0.206935
8,101.192500,-1.083732e-02,-0.001844,-0.009101,0.185000


In [133]:
getMeHourdata(15)

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,100.119986,NaN,NaN,NaN,0.010625
1,100.076029,-0.043904,-0.041379,-0.002527,0.010319
2,100.000213,-0.075759,-0.080033,0.004274,0.010427
3,99.969048,-0.031165,-0.027950,-0.003214,0.010000
4,99.954277,-0.014775,-0.025331,0.010552,0.009974
5,99.952841,-0.001437,0.006915,-0.008348,0.010103
6,100.012761,0.059949,0.059923,0.000026,0.010067
7,100.067342,0.054573,0.053675,0.000899,0.010227
8,100.047273,-0.020055,-0.024812,0.004759,0.010285


In [136]:
del finalDF['X']
del finalDF['x']
del finalDF['y']
del finalDF['i']


KeyError: '1'

In [137]:
del finalDF['q']

In [139]:
finalDF.to_csv("20200221Data.csv")